In [ ]:
!pip install -q accelerate bitsandbytes rich transformers --progress-bar off

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig

In [ ]:
import pickle

In [ ]:
experiment_name = "2024.09.23-03_AnalisiSentimentConPhi3-4k-mini"

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/../AnalisiTesti/data/output/full_data___6400_texts___sentiment_sdgwashing_sdgsmearing.xlsx"

output_data_path = base_path + "/../AnalisiTesti/data/output/" + experiment_name
log_filename = base_path + "/../AnalisiTesti/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

In [ ]:
import glob

def update_data(data_sel):
  drive.mount(mount_point, force_remount=True)

  results = []
  files = glob.glob(output_data_path + f"/*.pkl")
  for file in files:
    res = pd.read_pickle(file)
    results.append(res)
  print(len(results))

  df = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'output'])
  processed_texts = df.text_id.to_list()
  print(processed_texts)

  before = data_sel.shape[0]
  print("Full dataset size:               ", before)
  data = data_sel[~data_sel.text_id.isin(processed_texts)].copy() # DEEP COPY
  after = data.shape[0]
  print("Amount of files to be processed: ", after)
  before - after

  return data.sample(frac=1.00)

# data_sel = update_data(data_sel)
# data_sel.shape[0]

## Run the model

In [ ]:
data_sel = data

In [ ]:
results = []

while 1:

  data_sel = update_data(data_sel) # AGGIORNO IL DATAFRAME CON I DATI DA PROCESSARE A OGNI STEP

  if data_sel.shape[0]==0:
    break


  for index, row in data_sel.iterrows():

    cnt = index

    text_id = row['text_id']
    text = row["text"].replace('\n','').replace('{','(').replace('}',')')
    sdg_true = row["sdg_true"]

    file = open(log_filename, 'a+', encoding='utf-8')

    print("\n\n##################################################")
    file.write("\n\n##################################################\n")

    print("TEXT:", row["text"].replace('\n',''),'\n\n')
    file.write(f"TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  # The following are """+str(num_shots)+""" examples of expected classification given an input text:

  # """+few_shots+"""

    template = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

    - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

    - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

    - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

  Return your answer in the following "list of strings" format

    [LABEL, LABEL_JUSTIFICATION_1, LABEL_JUSTIFICATION_2, LABEL_JUSTIFICATION_3, ...]

  where the first element must be the value of the label ("positive", "negative", or "neutral") and the subsequent values must contain the texts related to the features that justify the provided output label.

  Do NOT add any of your comments outside the list.

  The input text with its label are:
  '''
  INPUT TEXT: \""""+text+"""\"
  SDG_CLASSIFICATION: "SDG-"""+str(sdg_true)+"""\"
  OUTPUT:
  '''
  """

    # print(template)

    messages = [
      {"role": "system", "content": "You are a text classifier."},
      {"role": "user", "content": template}
    ]

    output = pipe(messages, **generation_args)

    result_extended = (cnt, text_id, sdg_true, output[0]['generated_text'])

    print(result_extended)
    file.write(str(result_extended)+'\n')

    with open(output_data_path+f'/sdg_classification_{text_id}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)

    file.close()

    break # PER FORZARE L'USCITA DAL CICLO INTERNO E PERMETTERE L'AGGIORNAMENTO DEL DATAFRAME

Mounted at /content/gdrive
2336
[3402, 5311, 2158, 416, 2480, 5788, 3476, 4833, 2524, 2953, 1013, 5401, 1461, 3225, 1162, 5410, 2066, 1735, 3096, 4036, 1695, 3799, 4902, 1312, 646, 4149, 6202, 4046, 3548, 4547, 5225, 1517, 357, 4221, 3939, 3715, 2164, 5275, 4554, 3180, 2608, 5405, 6792, 2611, 553, 3354, 951, 6396, 6617, 561, 2501, 5976, 3469, 5864, 594, 1928, 1296, 1721, 6775, 6399, 6264, 5469, 184, 1924, 2233, 2377, 2723, 1028, 6209, 3045, 4683, 4454, 4182, 4623, 1283, 1430, 1772, 4259, 3492, 1359, 5914, 6087, 1736, 2142, 3924, 1012, 6120, 1746, 6014, 4967, 2535, 566, 2580, 5673, 413, 2045, 1645, 1586, 6568, 447, 2101, 226, 113, 4575, 2763, 4028, 4846, 5467, 2041, 619, 155, 1443, 5274, 2426, 1379, 5982, 1116, 4244, 6197, 2217, 2347, 89, 537, 4467, 1508, 494, 1773, 2971, 5644, 122, 6643, 3887, 5365, 5368, 3293, 6663, 45, 5253, 71, 6654, 3607, 1220, 6603, 1853, 3219, 727, 2155, 1122, 3347, 759, 2813, 3706, 6061, 4372, 6314, 4803, 818, 2076, 3565, 4963, 409, 6741, 593, 4316, 3269, 625, 2

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


(2069, 2491, 5, ' ["positive", "The text states that at least 90 countries apply an electoral gender quota for their national parliaments, which is a measure to promote gender equality and empower women, aligning with SDG-5.", "The text mentions that 16 countries have reserved seats for women, 33 have legislated candidate quotas, and 54 have voluntary political party quotas, all of which contribute to advancing the goal of gender equality.", "The text does not imply any negative impact on SDG-5, as all mentioned measures are aimed at increasing women\'s representation in political bodies."]')
Mounted at /content/gdrive
2338
[3402, 5311, 2158, 416, 2480, 5788, 3476, 4833, 2524, 2953, 1013, 5401, 1461, 3225, 1162, 5410, 2066, 1735, 3096, 4036, 1695, 3799, 4902, 1312, 646, 4149, 6202, 4046, 3548, 4547, 5225, 1517, 357, 4221, 3939, 3715, 2164, 5275, 4554, 3180, 2608, 5405, 6792, 2611, 553, 3354, 951, 6396, 6617, 561, 2501, 5976, 3469, 5864, 594, 1928, 1296, 1721, 6775, 6399, 6264, 5469, 18

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


(1746, 3457, 4, ' ["positive", "Authorities are aware of the discrepancies, and have started to increase investment.", "Over the 2007-11 period 8 000 new classrooms are scheduled to be built and subsidised, of which 40% are for the benefit of minority populations.", "Spending on educational institutions as a share of GDP has decreased from 8.5% to 7.5%", "Budgetary spending per child (aged 0-17) in Arab localities is only half of Jewish communities’ expenditure per child (Sikkuy, 2008)"]')
Mounted at /content/gdrive
2339
[3402, 5311, 2158, 416, 2480, 5788, 3476, 4833, 2524, 2953, 1013, 5401, 1461, 3225, 1162, 5410, 2066, 1735, 3096, 4036, 1695, 3799, 4902, 1312, 646, 4149, 6202, 4046, 3548, 4547, 5225, 1517, 357, 4221, 3939, 3715, 2164, 5275, 4554, 3180, 2608, 5405, 6792, 2611, 553, 3354, 951, 6396, 6617, 561, 2501, 5976, 3469, 5864, 594, 1928, 1296, 1721, 6775, 6399, 6264, 5469, 184, 1924, 2233, 2377, 2723, 1028, 6209, 3045, 4683, 4454, 4182, 4623, 1283, 1430, 1772, 4259, 3492, 1359, 

...

(1430, 4634, 2, ' ["positive", "The PAA supports smallholder farmers by purchasing their products, which contributes to food security and aligns with SDG-2\'s aim to end hunger and promote sustainable agriculture.", "The PAA\'s requirement for public schools to buy from small producers ensures that a portion of the food served is sourced sustainably, furthering the goal of SDG-2.", "The National School Feeding Programme (PNAE) mandates that a significant percentage of food in public schools comes from small producers, which supports the sustainable agriculture aspect of SDG-2."]')
Mounted at /content/gdrive
2658
[3402, 5311, 2158, 416, 2480, 5788, 3476, 4833, 2524, 2953, 1013, 5401, 1461, 3225, 1162, 5410, 2066, 1735, 3096, 4036, 1695, 3799, 4902, 1312, 646, 4149, 6202, 4046, 3548, 4547, 5225, 1517, 357, 4221, 3939, 3715, 2164, 5275, 4554, 3180, 2608, 5405, 6792, 2611, 553, 3354, 951, 6396, 6617, 561, 2501, 5976, 3469, 5864, 594, 1928, 1296, 1721, 6775, 6399, 6264, 5469, 184, 1924, 223

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


(2780, 5638, 8, ' ["neutral", "The text does not explicitly state whether the strategies identified for improving skills governance lead to advancement or setback for SDG-8.", "The text does not provide any direct comparison or evaluation of the effectiveness of these strategies in relation to SDG-8.", "The text merely describes the process of collecting information on skills governance without indicating its impact on SDG-8."]')
Mounted at /content/gdrive
2661
[3402, 5311, 2158, 416, 2480, 5788, 3476, 4833, 2524, 2953, 1013, 5401, 1461, 3225, 1162, 5410, 2066, 1735, 3096, 4036, 1695, 3799, 4902, 1312, 646, 4149, 6202, 4046, 3548, 4547, 5225, 1517, 357, 4221, 3939, 3715, 2164, 5275, 4554, 3180, 2608, 5405, 6792, 2611, 553, 3354, 951, 6396, 6617, 561, 2501, 5976, 3469, 5864, 594, 1928, 1296, 1721, 6775, 6399, 6264, 5469, 184, 1924, 2233, 2377, 2723, 1028, 6209, 3045, 4683, 4454, 4182, 4623, 1283, 1430, 1772, 4259, 3492, 1359, 5914, 6087, 1736, 2142, 3924, 1012, 6120, 1746, 6014, 4967, 2

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


(5114, 2643, 7, ' ["neutral", "The text discusses the development of new technology in the energy sector, which is related to SDG-7 (Affordable and Clean Energy).", "The text does not explicitly state whether the development of new technology is advancing or hindering SDG-7.", "The text mentions that transitions in energy technologies take decades and require massive investments, which does not directly imply a positive or negative impact on SDG-7."]')


In [ ]:
STOP!!!

In [ ]:
import pandas as pd

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

In [ ]:
results_processed.iloc[0].result

In [ ]:
#results_processed.iloc[0].result.split("Answer with label only usign the format:---LABEL---")[1]
# results_processed['result_short'] = results_processed['result'].apply(lambda x: x.split("Answer with label only usign the format:---LABEL---")[1])

In [ ]:
# results_processed['result']

In [ ]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
# results_processed['label'] = results_processed['result_short'].str.extract(r'SDG (\d+)')

# results_processed['label'] = results_processed['result'].apply(
#     lambda x: pd.Series(x[0]['generated_text']).str.extract(r'SDG-(\d+)')[0]
# )

results_processed.head(3)

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final.head(3)

In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

In [ ]:
final  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test

In [ ]:
results_processed_merged = final

In [ ]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['label']) or row['label'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'label'] = 0  # new_value

In [ ]:
results_processed_merged.head(3)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
output_datapath = output_data_path

In [ ]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [ ]:
report_df

In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))